- http://hiyoko9t.hatenadiary.jp/entry/2017/10/05/210419

In [1]:
import tensorflow as tf

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # 暗にtf.float32を宣言
print(node1)
print(node2)

Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)


In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [7]:
# 事前に値を与えてからの計算グラフ
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [23]:
# 入力型として計算グラフの書き方
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # tf.add(a, b)のショートカット
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[3. 7.]


In [9]:
# 計算グラフを追加
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [64]:
# 変数として計算グラフを書く .3 = 0.3
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

# 変数処理を行うinitializerを初期化する
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

# y=wx+bとして計算している

[0.         0.3        0.6        0.90000004]


In [65]:
# 上記で計算した値と実際のyの二乗誤差を取る
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y) #各サンプルについて、二乗誤差を計算
loss = tf.reduce_sum(squared_deltas)#sumで畳み込む
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [70]:
#計算グラフの変数を変更する
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])

print(sess.run(linear_model, {x: [1, 2, 3, 4]}))
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

[ 0. -1. -2. -3.]
0.0


## 機械学習による最適化をやってみる

In [89]:
#オプティマイザー
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [90]:
sess.run(init) # 変数のリセット
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([0.9999908], dtype=float32)]


# 単回帰モデルのまとめ

In [91]:
import tensorflow as tf

# モデルパラメータ
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# モデルの入出力
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# ロス
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# オプティマイザ
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# 訓練データ
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# 訓練ループ
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# 訓練精度の評価
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11


## tf.estimator を使うと、機械学習部分が簡単に書ける

In [92]:
import tensorflow as tf
# NumPy はデータのロード、計算、前処理によく用いられます。
import numpy as np

# 特徴量のリストの宣言。ここでは、一つの特徴量だけですが、他にも多くの、より複雑で有用な型が存在します。
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# 推定器による訓練と評価。線形回帰や線形分類器、ニューラルネットワーク回帰、分類器など多くの準備された型が存在します。(ここでは線形回帰を用います)
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TFは、多くのデータセットの読み込みやセットアップに役立つメソッドが準備されています。ここでは、訓練データと評価データを使用します。
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# 訓練データを用いて、訓練を1000ステップ実行
estimator.train(input_fn=input_fn, steps=1000)

# モデルの性能を評価
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_log_step_count_steps': 100, '_session_config': None, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f154e262dd8>, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_is_chief': True, '_task_type': 'worker', '_master': '', '_task_id': 0, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/tmptyhh2wpk'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmptyhh2wpk/model.ckpt.
INFO:tensorflow:step = 1, loss = 6.0
INFO:tensorflow:gl

# カスタムモデル

In [93]:
import numpy as np
import tensorflow as tf

# 特徴量の宣言
def model_fn(features, labels, mode):
  # 線形モデルと予測変数のビルド
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W * features['x'] + b
  # ロス
  loss = tf.reduce_sum(tf.square(y - labels))
  # 訓練
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # 推定器との接続
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# データセットの定義
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# 訓練
estimator.train(input_fn=input_fn, steps=1000)
# モデルの評価
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_log_step_count_steps': 100, '_session_config': None, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f154cb71a90>, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_is_chief': True, '_task_type': 'worker', '_master': '', '_task_id': 0, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/tmpz66prm2b'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpz66prm2b/model.ckpt.
INFO:tensorflow:step = 1, loss = 147.2298160517733
INFO